In [ ]:
deps_path = '/kaggle/input/czii-cryoet-dependencies'

In [ ]:
! cp -r /kaggle/input/czii-cryoet-dependencies/asciitree-0.3.3/ asciitree-0.3.3/

In [ ]:
! pip wheel asciitree-0.3.3/asciitree-0.3.3/

In [ ]:
!pip install asciitree-0.3.3-py3-none-any.whl

In [ ]:
! pip install -q --no-index --find-links {deps_path} --requirement {deps_path}/requirements.txt

In [ ]:
# pip installがされたかの確認
!pip show monai

In [ ]:
from typing import List, Tuple, Union
import numpy as np
import torch
from monai.data import DataLoader, Dataset, CacheDataset, decollate_batch
from monai.transforms import (
    Compose,
    EnsureChannelFirstd,
    Orientationd,
    AsDiscrete,
    RandFlipd,
    RandRotate90d,
    NormalizeIntensityd,
    RandCropByLabelClassesd,
)

In [ ]:
# 指定された次元を完全に覆うために最小限の重複でパッチの開始位置を計算する
def calculate_patch_starts(dimension_size: int, patch_size: int) -> List[int]:
    """
    Calculate the starting positions of patches along a single dimension
    with minimal overlap to cover the entire dimension.

    Parameters:
    -----------
    dimension_size : int
        Size of the dimension
    patch_size : int
        Size of the patch in this dimension

    Returns:
    --------
    List[int]
        List of starting positions for patches
    """
    if dimension_size <= patch_size:
        return [0]

    # Calculate number of patches needed
    n_patches = np.ceil(dimension_size / patch_size)

    # ここのコードはいらない気もするが一用残しておく
    if n_patches == 1:
        return [0]

    # Calculate overlap
    total_overlap = (n_patches * patch_size - dimension_size) / (n_patches - 1)

    # Generate starting positions
    positions = []
    for i in range(int(n_patches)):
        pos = int(i * (patch_size - total_overlap))
        if pos + patch_size > dimension_size:
            pos = dimension_size - patch_size
        if pos not in positions:  # Avoid duplicates
            positions.append(pos)

    return positions

def extract_3d_patches_minimal_overlap(arrays: List[np.ndarray], patch_size: int) -> Tuple[List[np.ndarray], List[Tuple[int, int, int]]]:
    """
    Extract 3D patches from multiple arrays with minimal overlap to cover the entire array.
    複数の3D配列から最小限の重複を持つバッチを抽出し、全体をカバーします
    
    Parameters:
    -----------
    arrays : List[np.ndarray]
        List of input arrays, each with shape (m, n, l)
        抽出する立方体パッチのサイズ(a x a)
    patch_size : int
        Size of cubic patches (a x a x a)
        抽出する立方体パッチサイズ
        
    Returns:
    --------
    patches : List[np.ndarray]
        List of all patches from all input arrays
        全ての入力配列からちゅしゅつされたパッチのリスト
    coordinates : List[Tuple[int, int, int]]
        List of starting coordinates (x, y, z) for each patch
        各パッチの開始位置
    """
    # 入力が非空のリストであることを確認
    if not arrays or not isinstance(arrays, list):
        raise ValueError("Input must be a non-empty list of arrays")

    # 全ての配列が同じ形状を持つ配列であることを確認
    # Verify all arrays have the same shape
    shape = arrays[0].shape
    if not all(arr.shape == shape for arr in arrays):
        raise ValueError("All input arrays must have the same shape")

    # パッチサイズが各次元の最小サイズより小さいことを確認
    if patch_size > min(shape):
        raise ValueError(f"patch_size ({patch_size}) must be smaller than smallest dimension {min(shape)}")
    
    m, n, l = shape
    patches = [] # 抽出されたパッチを格納するリスト
    coordinates = [] # 各パッチの開始座標を格納するリスト
    
    # Calculate starting positions for each dimension
    # 各次元に対するパッチの開始位置を計算
    x_starts = calculate_patch_starts(m, patch_size)
    y_starts = calculate_patch_starts(n, patch_size)
    z_starts = calculate_patch_starts(l, patch_size)
    
    # Extract patches from each array
    # 各配列からパッチを抽出
    for arr in arrays:
        for x in x_starts:
            for y in y_starts:
                for z in z_starts:
                    # 配列からパッチを切り出し
                    patch = arr[
                        x:x + patch_size,
                        y:y + patch_size,
                        z:z + patch_size
                    ]
                    patches.append(patch)
                    coordinates.append((x, y, z))
    
    return patches, coordinates # パッチのリストと座標のリストを返す


